In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime, time
import sklearn, sklearn.metrics, sklearn.preprocessing, sklearn.linear_model, sklearn.ensemble

import modutils

#data_dir = '../DataSets/Churn/'
data_dir = '../DataSets/kkbox/churn/raw/'
logm_fmt = data_dir + 'user_logs/uldtm_{0}.csv'
used_log = '201701'

In [2]:
src0 = pd.read_csv(logm_fmt.format(used_log))

In [3]:
src0['lag0_used'] = src0.used
src0['lag1_used'] = src0.used.shift(1)
src0['lag2_used'] = src0.used.shift(2)
src0['lag3_used'] = src0.used.shift(3)
src0['lag4_used'] = src0.used.shift(4)

In [7]:
src0['hist5'] = src0.lag0_used.map(lambda x: '1' if x > 0.5 else '0')+\
                src0.lag1_used.map(lambda x: '1' if x > 0.5 else '0')+\
                src0.lag2_used.map(lambda x: '1' if x > 0.5 else '0')+\
                src0.lag3_used.map(lambda x: '1' if x > 0.5 else '0')+\
                src0.lag4_used.map(lambda x: '1' if x > 0.5 else '0')

In [9]:
src0[src0.date>20170105].groupby('hist5').nxt_used.mean()

hist5
00000    0.092840
00001    0.220103
00010    0.216622
00011    0.328110
00100    0.225498
00101    0.366406
00110    0.344637
00111    0.463841
01000    0.257029
01001    0.405217
01010    0.391034
01011    0.509740
01100    0.381079
01101    0.516219
01110    0.490762
01111    0.621925
10000    0.410523
10001    0.524124
10010    0.496161
10011    0.643285
10100    0.500432
10101    0.618735
10110    0.604275
10111    0.747586
11000    0.563535
11001    0.670687
11010    0.629685
11011    0.769248
11100    0.668197
11101    0.771152
11110    0.753685
11111    0.888481
Name: nxt_used, dtype: float64

In [12]:
hist5onehot = sklearn.preprocessing.OneHotEncoder().fit(src0.hist5.values.reshape(-1,1))

In [13]:
X = hist5onehot.transform(src0.hist5[src0.date>20170105].values.reshape(-1,1))

In [15]:
Y = src0.nxt_used[src0.date>20170105].values

In [17]:
X.shape, Y.shape

((29536078, 32), (29536078,))

In [20]:
m_logreg = sklearn.linear_model.LogisticRegression().fit(X[~np.isnan(Y)], Y[~np.isnan(Y)])

In [21]:
pred_p = m_logreg.predict_proba(X[~np.isnan(Y)])[:,1]

In [23]:
pred_lgt = np.log(pred_p/(1-pred_p))

In [25]:
sklearn.metrics.roc_auc_score(Y[~np.isnan(Y)], pred_lgt)*2-1

0.70187531922614799

In [27]:
Xd = X[~np.isnan(Y)]
Yd = Y[~np.isnan(Y)]

In [28]:
(Xt,Yt),(Xv,Yv) = modutils.splitSample((Xd,Yd),pcts=[0.4,0.6])

In [29]:
m_gradboost = sklearn.ensemble.GradientBoostingClassifier().fit(Xt, Yt)

In [30]:
Pv = m_logreg.predict_proba(Xv)[:,1]
Sv = np.log(Pv/(1-Pv))
sklearn.metrics.roc_auc_score(Yv, Sv)*2-1

0.70178393982551146